In [6]:
import time

import torch
from torch import Tensor
torch.manual_seed(0)
from torch import nn
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
stdscaler = StandardScaler()

def closure():
        optimizer.zero_grad()
        y_hat = model.forward(X_train)
        loss = criterion(y_hat, y_train)
        loss.backward()
        return loss
    
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()

import numpy as np
from sklearn.model_selection import GridSearchCV

from copy import deepcopy

data = pd.read_csv('Proj1ReorderedRound.csv')

import warnings
warnings.filterwarnings('ignore')

In [7]:
X = data.drop('Condition', axis=1).values
y = data['Condition'].values

y = LE.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train = stdscaler.fit_transform(X_train)
X_test = stdscaler.transform(X_test)
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [3]:
class ANN(nn.Module):
    def __init__(self,af, array):
        super().__init__()
        
        self.fc1 = nn.Linear(in_features = 12, out_features = array[0])
        self.output = nn.Linear(in_features=array[0], out_features=3)
        
        self.AF = af
 
    def forward(self, x):
            
        if (af == 'sigmoid'):
            x = F.sigmoid(self.fc1(x))
            x = self.output(x)
            
        if (af == 'tanh'):
            x = F.tanh(self.fc1(x))
            x = self.output(x)
        
        if (af == 'relu'):
            x = F.relu(self.fc1(x))
            x = self.output(x)
            
        if (af == 'leaky_relu'):
            x = F.leaky_relu(self.fc1(x))
            x = self.output(x)
            
        if (af == 'elu'):
            x = F.elu(self.fc1(x))
            x = self.output(x)
            
        if (af == 'softmax'):
            x = F.softmax(self.fc1(x))
            x = self.output(x)
            
        if (af == 'hardswish'):
            x = F.hardswish(self.fc1(x))
            x = self.output(x)
            
        if (af == 'gelu'):
            x = F.gelu(self.fc1(x))
            x = self.output(x)
            
        if (af == 'selu'):
            x = F.selu(self.fc1(x))
            x = self.output(x)
        
        return x
    
alpha = 0.1
criterion = nn.CrossEntropyLoss()

In [4]:
initiator = 0

two_thirds = [12,8,5,3,2]

ones = []
threes = []
halves = []
fulls = []
desc = []
asc = []

config_list = []


for i in range(1):
    ones.append(1)
    threes.append(3)
    halves.append(6)
    fulls.append(12)
    desc.append(two_thirds[i])
    asc.append(two_thirds[:3][-1-i])

configDF = pd.DataFrame({'ones':ones,'threes':threes,'halves':halves,'fulls':fulls,'desc':desc,'asc':asc})
print('# of hidden layers : ', 2)
configDF

# of hidden layers :  5


,ones,threes,halves,fulls,desc,asc
0,1,3,6,12,12,2
1,1,3,6,12,8,3
2,1,3,6,12,5,5
3,1,3,6,12,3,8
4,1,3,6,12,2,12


In [5]:
time_list = []

epochs = 100
print('Layers : ', layers)
print('Epochs : ', epochs)

LossDF = pd.DataFrame(columns = ['config','nodes','initial','midway','final'])
index = 0

if (initiator == 0):    #checks if this cell has been run before...
    FML = []   # Full Model List : stores initial model weights and bias for future reference
      
for config in configDF.columns:
    Start = time.time()     #start time
    
    to_add = [config]
    
    Arr = configDF[config]
    to_add.append(Arr.sum())
    
    if (initiator == 0):        # if this cell wasnt run before, store the starting weights and bias for future ref
        model = ANN(layers,Arr)
        dummy = deepcopy(model)
        FML.append(dummy)
        
    elif (initiator == 1):      # otherwise, use the same starting params...
        model = deepcopy(FML[index])
        setattr(model, 'array', Arr)    #... while overwriting the hidden layer configuration
        
    optimizer = torch.optim.SGD(model.parameters(), lr=alpha) # LBFGS
    
    for e in range(epochs):
        y_hat = model.forward(X_train)
        loss = criterion(y_hat, y_train)
                
        if e == 0:
            to_add.append(loss.detach().numpy())
                    
        if e == 49:
            to_add.append(loss.detach().numpy())
 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
    to_add.append(loss.detach().numpy())
            
    LossDF.loc[index] = to_add
    index += 1
    
    
    End = time.time()
    dura = End - Start
    time_list.append(dura)
    
    print('\nComputation for',config,'complete')
    print('Time taken : ', np.round(dura,3),'s')

initiator = 1

print('\nTotal time taken :',np.round(np.sum(time_list),3),'s')

print('average loss = ', np.round(LossDF.mean()[-1],5))
print('average loss (without ones)= ', np.round(LossDF.drop(0).mean()[-1],5))

LossDF

Layers :  5
Epochs :  100

Computation for ones complete
Time taken :  0.067 s

Computation for threes complete
Time taken :  0.07 s

Computation for halves complete
Time taken :  0.074 s

Computation for fulls complete
Time taken :  0.075 s

Computation for desc complete
Time taken :  0.072 s

Computation for asc complete
Time taken :  0.068 s

Total time taken : 0.426 s
average loss =  1.09102
average loss (without ones)=  1.09047


,config,nodes,initial,midway,final
0,ones,5,1.2038121,1.0956242,1.0937479
1,threes,15,1.1463084,1.0930166,1.0914633
2,halves,30,1.10634,1.0892259,1.0796334
3,fulls,60,1.105646,1.0944269,1.093803
4,desc,30,1.103818,1.0940338,1.0937507
5,asc,30,1.1099824,1.0939381,1.0937195
